# 4. Knowledge Distilation (Working Title)

(Working Summary) Knowledge distillation is conducted by using a large model to train a small model.

Code based on tutorial from Keras
https://keras.io/examples/vision/knowledge_distillation/



In [2]:
import os

import keras
from keras import layers
from keras import ops
import numpy as np

In [3]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def compute_loss(
        self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False
    ):
        teacher_pred = self.teacher(x, training=False)
        student_loss = self.student_loss_fn(y, y_pred)

        distillation_loss = self.distillation_loss_fn(
            ops.softmax(teacher_pred / self.temperature, axis=1),
            ops.softmax(y_pred / self.temperature, axis=1),
        ) * (self.temperature**2)

        loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
        return loss

    def call(self, x):
        return self.student(x)

In [4]:
# Create the teacher
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)

# Create the student
student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

# Clone student for later comparison
student_scratch = keras.models.clone_model(student)

In [5]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Train teacher as usual
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate teacher on data.
teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 589s 313ms/step - loss: 0.2459 - sparse_categorical_accuracy: 0.9209
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 596s 318ms/step - loss: 0.0897 - sparse_categorical_accuracy: 0.9728
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 590s 315ms/step - loss: 0.0783 - sparse_categorical_accuracy: 0.9769
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 613s 310ms/step - loss: 0.0694 - sparse_categorical_accuracy: 0.9794
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 592s 315ms/step - loss: 0.0614 - sparse_categorical_accuracy: 0.9815
313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 91ms/step - loss: 0.1363 - sparse_categorical_accuracy: 0.9649


[0.12787702679634094, 0.968500018119812]

In [7]:
# Initialize and compile distiller
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 603s 321ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.7450
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 618s 319ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.9482
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 617s 316ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.9613
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 85ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.9600


[0.0, 0.0, 0.9671000242233276, 0.9671000242233276]

In [8]:
# Train student as doen usually
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - loss: 0.4793 - sparse_categorical_accuracy: 0.8569
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.1006 - sparse_categorical_accuracy: 0.9691
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0747 - sparse_categorical_accuracy: 0.9769
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0921 - sparse_categorical_accuracy: 0.9713


[0.07569274306297302, 0.9771000146865845]